In [1]:
import pandas as pd
import re
from lxml import etree
import shutil
import os
import math
import copy
import glob

In [2]:
def load_base_value_iden(file_path):
    if os.path.exists(file_path):
        
        # File exists, load the variable from the file
        if os.path.getsize(file_path) > 0:
                    # File is not empty, load the variable from the file
            with open(file_path, 'r') as file:
                
                variable_loaded = file.read().strip()
#                 print('loading')
#                 print(variable_loaded)
        else:
            # File exists but is empty, initialize with a default value
            variable_loaded = "11e1d6e4-74ba-49cc-9b56-2"
    else:
        # File does not exist, create the file and write the variable to it
#         print('+++++')
        variable_to_store = "11e1d6e4-74ba-49cc-9b56-2"
        with open(file_path, 'w') as file:
            file.write(variable_to_store)
        variable_loaded = variable_to_store
        
    return variable_loaded
def save_base_value_iden(file_path,variable_to_store):
    
    # Write the variable to the file
    with open(file_path, 'w') as file:
        file.write(variable_to_store)
def increment_value_iden(value,file_path):

    str_last = value.split('-')[-1]
    last= int(str_last)
    last+=1
    
    exclude_last = value.split('-')[:-1]
    temp = '-'.join(exclude_last)

    new_identifier = temp + '-' + str(last)
    
    if os.path.exists(file_path):
        with open(file_path, 'w') as file:
            variable_to_store = new_identifier
#             print('writing')
#             print(variable_to_store)
            file.write(variable_to_store)
    else:
        print('load base value first')

In [3]:
user_root = os.getcwd()
root_path = os.path.join(user_root,'iras_api_clean_1')

In [4]:
filepath_base_identifier = os.path.join(user_root,'instance_identifier.txt')
temp_file_path = filepath_base_identifier

In [5]:
import glob
def get_top_level_folders_with_numbers(root_path):
    directories = []
    folder_paths = glob.glob(os.path.join(root_path, '*/'))
    
    # Regular expression to check if the folder name starts with a number
    pattern = re.compile(r'^\d')

    for path in folder_paths:
        folder_name = os.path.basename(os.path.normpath(path))
        if pattern.match(folder_name):
            directories.append(path)
    return directories
def find_xml_files_directories(directories):
    xml_files_dirs = []
    for dir_path in directories:
        xml_files_paths = glob.glob(os.path.join(dir_path, 'xml_files*/'))
        xml_files_dirs.extend(xml_files_paths)
    return xml_files_dirs


directories_temp = get_top_level_folders_with_numbers(root_path)
directories = find_xml_files_directories(directories_temp)




# save_base_value_iden(temp_file_path,'13testing2-11aa-11aa-1a10-1')

In [6]:
directories

['/Users/cheongray/iras_api_make_xml/combine_all_copy/iras_api_clean_1/3_Code_Validation/xml_files_for_3_Code Validation/',
 '/Users/cheongray/iras_api_make_xml/combine_all_copy/iras_api_clean_1/4_Format_validation/xml_files_for_4_Format Validation/',
 '/Users/cheongray/iras_api_make_xml/combine_all_copy/iras_api_clean_1/5_Record_Validation/xml_files_for_5_Record Validation/',
 '/Users/cheongray/iras_api_make_xml/combine_all_copy/iras_api_clean_1/2_Conditional_Validation/xml_files_for_2_Conditional Input Validation/',
 '/Users/cheongray/iras_api_make_xml/combine_all_copy/iras_api_clean_1/6_Schema_Validation/xml_files_for_6_Schema Validation/',
 '/Users/cheongray/iras_api_make_xml/combine_all_copy/iras_api_clean_1/1_Input_Validation/xml_files_for_1_Input Validation/',
 '/Users/cheongray/iras_api_make_xml/combine_all_copy/iras_api_clean_1/7_Bulk_Submission/xml_files_for_7_Bulk Submission/']

In [7]:
def get_id(xml_file):
    temp = xml_file.split('/')[-1]
    temp_2 = temp.split('_Test_case_')[-1].split('_')[0]
    return temp_2


def extract_type(filepath_temp):
    name = filepath_temp.split('_')[-2]
    return name

def parse_xml(filepath_base_identifier):
    # Parse the XML file
    tree = etree.parse(filepath_base_identifier)
    # Get the root element
    root = tree.getroot()
    
    # Process the XML tree without printing
    return root    
def process_xml_files_in_folder_Invoice_ID(folder_path):
    
    # Use glob to find all XML files in the specified folder
#     namespace_map = {
#     'sb': 'http://www.unece.org/cefact/namespaces/StandardBusinessDocumentHeader',
#     'xs': 'http://www.w3.org/2001/XMLSchema',
#     'cac': 'urn:oasis:names:specification:ubl:schema:xsd:CommonAggregateComponents-2',
#     'cbc': 'urn:oasis:names:specification:ubl:schema:xsd:CommonBasicComponents-2',
#     }
#     reference = load_base_value_iden(temp_file_path)
#     print(f'ref:{reference}')
#     xpath_target = '/*/*[1]/*[4]/*[3]'
    xml_files = glob.glob(os.path.join(folder_path, '*.xml'))
    template = 'F12345_TestCaseID_'
    c=0
    count=0
    for index_file,xml_file in enumerate(xml_files):
#         print('---')
        count+=1
        root = parse_xml(xml_file)
        base_identifier = load_base_value_iden(temp_file_path)
        file_type = extract_type(xml_file)
        test_case_id = get_id(xml_file)
        element_array = []
        tree = etree.ElementTree(root)
        for index,e in enumerate(root.iter()):   
            element_array.append(e)
            if index == 10:
#                 print('_________')
#                 print(e.text)
#                 print('_________')
                hold_text = str(e.text)
                #if dont have '-' means its not the right element,but could also mean supposed to be empty or not there
                if '-' not in hold_text:
                    if e.text == None:
                        c+=1
#                         print('keep',xml_file)
                        continue

                    if e.text == 'nan':
                        c+=1
                        continue

                    else:
                        
                        
                        e = element_array[index-1]

#                         print('xml file:',xml_file)
                        e.text = base_identifier
                        increment_value_iden(base_identifier,temp_file_path) 
                        tree.write(xml_file, encoding='utf-8', xml_declaration=True)
                else:
                    e.text = base_identifier

#                     print('xml file:',xml_file)
                    increment_value_iden(base_identifier,temp_file_path)
                    tree.write(xml_file, encoding='utf-8', xml_declaration=True)
    return count

In [21]:
for filefolderpath in directories:
    print(filefolderpath)
    print('-------')
    num_files = process_xml_files_in_folder_Invoice_ID(filefolderpath)
    final_value = load_base_value_iden(temp_file_path)
    print('final base value ',final_value)
    print(' ')

/Users/cheongray/iras_api_make_xml/combine_all_copy/iras_api_clean_1/3_Code_Validation/xml_files_for_3_Code Validation/
-------
final base value  111AUG-11aa-11aa-1a10-217399
 
/Users/cheongray/iras_api_make_xml/combine_all_copy/iras_api_clean_1/4_Format_validation/xml_files_for_4_Format Validation/
-------
final base value  111AUG-11aa-11aa-1a10-218167
 
/Users/cheongray/iras_api_make_xml/combine_all_copy/iras_api_clean_1/5_Record_Validation/xml_files_for_5_Record Validation/
-------
final base value  111AUG-11aa-11aa-1a10-218687
 
/Users/cheongray/iras_api_make_xml/combine_all_copy/iras_api_clean_1/2_Conditional_Validation/xml_files_for_2_Conditional Input Validation/
-------
final base value  111AUG-11aa-11aa-1a10-218903
 
/Users/cheongray/iras_api_make_xml/combine_all_copy/iras_api_clean_1/6_Schema_Validation/xml_files_for_6_Schema Validation/
-------
final base value  111AUG-11aa-11aa-1a10-219015
 
/Users/cheongray/iras_api_make_xml/combine_all_copy/iras_api_clean_1/1_Input_Valida

In [410]:
#need change the uuid and invoice id
inovice_id_identifier ='12345'
def process_xml_files_in_folder_Invoice_ID(folder_path):
    
    # Use glob to find all XML files in the specified folder
#     namespace_map = {
#     'sb': 'http://www.unece.org/cefact/namespaces/StandardBusinessDocumentHeader',
#     'xs': 'http://www.w3.org/2001/XMLSchema',
#     'cac': 'urn:oasis:names:specification:ubl:schema:xsd:CommonAggregateComponents-2',
#     'cbc': 'urn:oasis:names:specification:ubl:schema:xsd:CommonBasicComponents-2',
#     }
#     reference = load_base_value_iden(temp_file_path)
#     print(f'ref:{reference}')
#     xpath_target = '/*/*[1]/*[4]/*[3]'
    xml_files = glob.glob(os.path.join(folder_path, '*.xml'))
    template = 'F12345_TestCaseID_'
    c=0
    for index_file,xml_file in enumerate(xml_files):
        root = parse_xml(xml_file)
#         file_type = extract_type(xml_file)
        test_case_id = get_id(xml_file)
        element_array = []
        tree = etree.ElementTree(root)
        for index,e in enumerate(root.iter()):   
            element_array.append(e)
            if index == 35:
                if 'F012345' == str(e.text):
                    pass
                else:
                     if 'BT' in str(e.text):
                            new_e = element_array[index-1]
                            if 'F012345' != str(new_e.text):
                                print(f'new text : {new_e.text},{xml_file}')
#                 if e.text == None:
#                     pass
#                 elif inovice_id_identifier not in str(e.text):
#                     if 'INV' not in str(e.text):#inv is valid values
                        
# #                         print(e.text,xml_file)
                        
#                         new_e = element_array[index-1]
#                         print(f'new text : {new_e.text},{xml_file}')
#                         if '12345' not in str(new_e):
#                             pass
#                         else:
#                             insert_val = template + test_case_id
#                             print(f'new insert text : {e.text},{xml_file},{insert_val}')
#                             print('---------------------------------')

In [411]:
process_xml_files_in_folder_Invoice_ID(filefolderpath)

new text :  BT-23 ,/Users/cheongray/iras_api_make_xml/input_val_2/IRASC5-007_Test_case_220_FileNumber_220_r_.xml
new text :   BT-23  ,/Users/cheongray/iras_api_make_xml/input_val_2/IRASC5-007_Test_case_204_FileNumber_204_r_.xml
new text :   BT-23  ,/Users/cheongray/iras_api_make_xml/input_val_2/IRASC5-007_Test_case_200_FileNumber_200_r_.xml
new text :  BT-23 ,/Users/cheongray/iras_api_make_xml/input_val_2/IRASC5-007_Test_case_224_FileNumber_224_r_.xml
new text :  BT-23 ,/Users/cheongray/iras_api_make_xml/input_val_2/IRASC5-007_Test_case_216_FileNumber_216_r_.xml
new text :   BT-23  ,/Users/cheongray/iras_api_make_xml/input_val_2/IRASC5-007_Test_case_208_FileNumber_208_r_.xml
new text :  BT-23 ,/Users/cheongray/iras_api_make_xml/input_val_2/IRASC5-007_Test_case_212_FileNumber_212_r_.xml
new text :   BT-23  ,/Users/cheongray/iras_api_make_xml/input_val_2/IRASC5-007_Test_case_196_FileNumber_196_r_.xml
